In [1]:
import numpy as np

%config Completer.use_jedi = False

np.seterr(divide="ignore", invalid="ignore")


import math
import os
import sys

import ipywidgets as widgets
import pandas as pd
import pydicom as dc
from highcharts import Highchart
from IPython.display import HTML, Latex, Markdown, clear_output, display
from pydicom import dcmread
from pydicom.filebase import DicomBytesIO
from scipy import fftpack, stats
from skimage import color
from skimage.feature import canny
from skimage.transform import hough_circle, hough_circle_peaks

In [2]:
uploader = widgets.FileUpload(
    accept="",  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True,  # True to accept multiple files upload else False
)

In [3]:
display(uploader)

FileUpload(value={}, description='Upload', multiple=True)

In [7]:
def BusquedaCirculos(image, HoughParameters, PixelSize):
    edges = canny(
        image,
        sigma=HoughParameters[0],
        low_threshold=HoughParameters[1],
        high_threshold=HoughParameters[2],
    )
    hough_radii = np.array(HoughParameters[3])
    hough_radii = (hough_radii / PixelSize).astype(np.int)
    hough_res = hough_circle(edges, hough_radii)
    return hough_circle_peaks(
        hough_res, hough_radii, total_num_peaks=HoughParameters[4]
    )


def createCircularMask(h, w, center=None, radius=None):

    if center is None:  # use the middle of the image
        center = [int(w / 2), int(h / 2)]
    if radius is None:  # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w - center[0], h - center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0]) ** 2 + (Y - center[1]) ** 2)

    mask = dist_from_center <= radius
    return mask


def NormalizeMTF(Matrix, Agua, Aire):
    return (Matrix - Aire) / (Agua - Aire)


class Point:
    def __init__(self, x_init, y_init):
        self.x = x_init
        self.y = y_init

    def shiftX(self, d):
        return Point(self.x + d, self.y)

    def shiftY(self, d):
        return Point(self.x, self.y + d)

    def __repr__(self):
        return "".join(["Point(", str(self.x), ",", str(self.y), ")"])

    def __add__(a, b):
        return Point(a.x + b.x, a.y + b.y)

    def __getitem__(self, i):
        return Point(self.x[i], self.y[i])


# TC018
TC018_HoughParameters = [3, 0, 50, [99, 100], 1]
TC018_Limits = [87.5, 112.5]
TC018_OverResolution = 0.1
TC018_MaxAngle = 6
TC018_StepAngle = 0.5
TC018_Angles = [0, 30, 60, 90, 120, 150, 180]
Catphan_ExtRadius = 100  # radio externo del catphan

TrafficLightGreen = "url(data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAA8AAAAhCAIAAACJEmZbAAAABGdBTUEAALGPC/xhBQAAACBjSFJNAAB6JgAAgIQAAPoAAACA6AAAdTAAAOpgAAA6mAAAF3CculE8AAAABmJLR0QA/wD/AP+gvaeTAAAAB3RJTUUH4gQQCyENzLpnyQAAA6BJREFUOMtllN9qVFcUxr+19j7/ZjL/4iQhQQNiCC1oay1FMIVeeFFrL/oChV4U+hJC8Q36MPW62F60ITGEGkLBCIrGGmN00iQzc2Zy9vp6cWYmovvq7LN+e+21vm/vLWYmIgB2dnY2Nzf7eU5VkgIoOTMzs7KyUq/XzUxVQZLknTt36vU6AAfUgBmggdFYXl6+d+8eyRACSN69e7cMtKdqn6aVy+Iuq/tE3KU0q6RpGbp//z5JPHv2rN1uA5jKsqUoWQKWRT+GfgRdAs5Xqlm1CuD69etm5h8+fHhwcACRLJieDhviWpQUIHAMeZkPjrKkD2xtbW1vb/vBYABARGKyBVmgppBy9ykgMzuCHAHF6Wm329VSEAIJ0IakEAMIEDCgAZm0q6o6+iRhlkCIcebxMHKM8Izu0AJExokJCCz3bi8Uk5UjWiAd8h+1IPSAgg526v3fkR4Nh2WpADzGOMhHDPuwC9BaHPWdf2pFJ+8JR+g7dDlxUTIdvUWvF+PUivxwVNAE95MORPT8XNqIusMAoyXKtOX2fXp4PJjgZ7mbtSTRbgGr1VzsURTo9kKjov1hXHpyRotoNbVKytnpqDal3oGBgxwvXof/+n4wGMmqY+Uli9luaKupUaoSqUSukrnFGTQqZ43pxALvWKuIqHAcCZAkkiy2kWgTmsbBMDgFbZyHBGnEce8DdwC8OuRJj56GggxEwQj26hB7HZZr3/FS0O1j4zH33xahX1i3KPrF8wOu7TCEiTnv6K0q+x37bcPNX7BaSw9fu39fFoCpiL3njqqYcXa2vfSdP0n3fIx5Y/Rnurs2DPygbjNOVWrL32Z5dU8HYscMeWh91l/8vCZw71cC4MKVRjd5kVQx84VU5xiG+mbH7Cifflp/c9ApL82I9hLHc8PaRVv6RqeX6WMAXPxKniwOTrbrbw5GZ3bipaZtO/8lZq/AeyCAAVGKi7c4d1UnJY/oYTgNrjd9CTCyPKUCK+BTWNItJRYRz9G1C/s7ffVKG/2gQRTDI3nyV7ecmpnOz887pwCe/47Hv8J7uAji4GKox4NfdH+LANIkazQaIHnjxgqASJOoIis/6w8P9KdH7vs//NUfI0CjKAZw8+ZNkiC5vr6eJMmZBRGyaRE5O0KVSmV1dZUkQggkNzc3b9++3Wo1kyxWcYCoapxGzVbz1te31tbWyjdWSJZv88nJycbGxu7ubuewk+d5FEXNRnNhYeHatWvnzp0rmf8BxVXVHXjMTCwAAAAldEVYdGRhdGU6Y3JlYXRlADIwMTgtMDQtMTZUMTE6MzM6MTMrMDA6MDAxayl4AAAAJXRFWHRkYXRlOm1vZGlmeQAyMDE4LTA0LTE2VDExOjMzOjEzKzAwOjAwQDaRxAAAAABJRU5ErkJggg==)"
TrafficLightYellow="url(data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAA8AAAAhCAIAAACJEmZbAAAABGdBTUEAALGPC/xhBQAAACBjSFJNAAB6JgAAgIQAAPoAAACA6AAAdTAAAOpgAAA6mAAAF3CculE8AAAABmJLR0QA/wD/AP+gvaeTAAAAB3RJTUUH4gQQCyEOVbM2cwAAA7ZJREFUOMttVFtPJFUQ/qrOme7p6Rl6WBhYLrKrgLgEVzcEJhoJ7guJvvjom6/+BP+D+0CMf0WyJiQYs8aERFg2Q4yLiRfWWVhAZmF6rt3nlA89FyCe9EOlq07VV99XdchaS0QAyuXy3tOn9UZDmAUggKwdHhpaLhZ937fWMjNERES+fvRopFAAoIhyQAHIEwEAMH/v3sbGhogYYyAi6+vriWPQ9+97mQXwAqn7pKZdz0unE9fW1paI4OTkZHJyEoDvedOOOwPMEr9DPEc8A0xmMhnfB7C4uBjHsd7f3y+XywDS1qp2Owc1KJQkDEFHzVbVSwMolUp7e3u62WyKCIhcKwFoAuyhgzgHeNZUgQsgjqJqtcrU7cYBhkEeSIDks0AAzncvMzMnFkRgbRokuHYIEJHEAMA9x2sRA6JuYiEQbEvxsTEAhOhGtP2NrbCkCAqixVqlSo6qtFvoVtBXix5Y8xJmCiqvUjXWL2DOWw1IH10/WkQAPfO29+at6mAWly1T3gcqQtSP1/1uwA8f5D+eu6i1ODIykotHltXjUvbPl+HN3ALcHc+9+0bYJnv3jjOYRb0h5eP24lv29MIPa7WE6B4Snp2gyVHz4YPM/Iz2PUYrPnvF3/3cLv2Te16r0TXcxONDWJ7XxfddnXUEJJEZ1/j8o3jrV37+FyVK6F6PAxk7N6V0WhtiYgAcsQ4CdWfYAMLo8k0AYF792/BTMJElEQhgRGILIwcvakljnejE+uGZ+f3vWJmWrbWl3rZh2+H2k315vGsAWLmiPBNOX9svv422t5tSqXOlHp3Xv/8x/uIb22rbHuUd3FagmH45MA+/wgcL8e0R/eKInjyLAKuYjE3GpKcOwVjJB7lgyv2pckaXVhMHt536aRQZ01OHewPrOm5u3KlGZ7kUsiwuGc41s4U0k/qfOcnm3TA6pxTcW6TTIpZaVYnilue7tbB+TUsCixPrjGTHyA3ACoD4MYXHsYQuwg5z3MVN7FivAG8QzJJsBCvJjsHN4yZuK9YidnOgq7smYAXRcYcHgKUzvNIMDRGJdNUSALCGamdRwoO1lkdHRx3HAdA4RXgE0tSpxyCNiz+odioAHMcNggAisrq6CsBJuUrTyHs0/SnNfkbTn3BhXjFzKpUCUCwWoyiCiOzs7Liu219QhVSGWFH/D9Hm5qaIwBgjIru7u2trawMDOa0VdZ8bpTmbza6srCRPpjGGJIHPfHl5ub29fXh4eF45bzYaOpXKB/mJiYmlpaWxsTERIaL/AL4M0TIMhG9IAAAAJXRFWHRkYXRlOmNyZWF0ZQAyMDE4LTA0LTE2VDExOjMzOjE0KzAwOjAw9MwX9gAAACV0RVh0ZGF0ZTptb2RpZnkAMjAxOC0wNC0xNlQxMTozMzoxNCswMDowMIWRr0oAAAAASUVORK5CYII=)"
TrafficLightRed="url(data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAA8AAAAhCAIAAACJEmZbAAAABGdBTUEAALGPC/xhBQAAACBjSFJNAAB6JgAAgIQAAPoAAACA6AAAdTAAAOpgAAA6mAAAF3CculE8AAAABmJLR0QA/wD/AP+gvaeTAAAAB3RJTUUH4gQQCyEOVbM2cwAAA6VJREFUOMtllE1vG1UUht9z7tyZsR1/JKnr0BQhtSrqgraolVykSqTqCoEELPgnbNjAAkXdsOJP8Ccq2OQDGQmRFBWptKuQKCRp0rQe22PPvfdlMU5C0+W55zlf77k6EkIQEQBbW1t/bm5meV6oetAAaWC73e7evVupVEIIqgqSJJeXly/MzwOoqH4AfAy9rgYAgJs3bjx69Iik9x4kHz58WDqutFrf1FrfIf5Wk+81+apan6lUS9fa2hpJ2d7evnP79t7+/myj8blDbZiJRinhgAn901bzV1+M+v179+6trq5Gm5ube/v7ELlCaQ6H1yS+Q70I9cAzODPInzarO/3+xsbGkydPojzPAcCYBfIW9FPYBSggAN4Xc8m5DWN3ADeZZFmmpSAQaUM+glmAKSAOKIACcovmphgAAqiqlkPAO04mHagHpPQBAYBgVBYHSJ7QgX94l4so6IEAOEFCv2P158kIAAEAU1ohj4P7QYuJMBFahiT4Q2u+TmRrNJSyDhBNqwhA/ujz31B8iehymv5j45/C+HGWKRmmDZzQU8NEf83Ff8uwWfEDrwcvvJCUkz7OcpMQvXyxMpcMCwfvXE0wO2d2++mrfo63c8/Wk8QM8+BrNWMjeM/B0DcrJp/E4/HkTVq0loRKEi7O2caMGCMIIR/JzoF7nSfjMeT/mhiVNOGFls611FaMxqo2qlXNe200qmeD6YnctIb1qoiRciQqPCSJpRoHlMs8pRk4KXykOFOLBBmIbOjO5waw9xKDEQ0DHOkJR4tw8Ar/HrOMPaNFkI34+3MeHTuOHcYujN3OIXvP6BxPM57qDRHsHYVfXmt7PlSqOhiY3X1HBBHhue2IgGSz2Wi8a1+ODvsRJQ51xIMXhQ/+fN8k0yRtLEZZcWgF6ikhmGbeaKcqer4TALXZuD85MjGSWYlSMGDcZ1GMK9V0MBi+QRs1tIWtof6OJA2IIYCak2zXh75g8KbegGjMahvpLEQJAoQazlxi0nprlz4EikvqOPudKANA48pViohyKg/zzIsKwglPAAhOBoeutEII2ul0VBXgcI/ZLsTKtJ4CEY6fy/AwAEiTtNFowHvf7XYBWBsbK50P5epncu0LufqJXrhuRNTaGMDS0hJJkFxfXz87FYBEsFVRI6cv1tqVlRWS8N6T7PV6Dx48qNdnoshM1QJMZOr1+v3791dXV8sbKyTL23x8fNzr9ba3t4+Ojkb5KLZxq9VaXFzsdrudTqdk/gNkQeRVLDIwDwAAACV0RVh0ZGF0ZTpjcmVhdGUAMjAxOC0wNC0xNlQxMTozMzoxNCswMDowMPTMF/YAAAAldEVYdGRhdGU6bW9kaWZ5ADIwMTgtMDQtMTZUMTE6MzM6MTQrMDA6MDCFka9KAAAAAElFTkSuQmCC)"


In [8]:
def LoadDicomFiles(X):
    output = []
    for dicom in X.value.keys():
        output.append(dcmread(DicomBytesIO(uploader.value[dicom]["content"])))
    return output

In [9]:
TC018_image = LoadDicomFiles(uploader)

| Código| Parámetro&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; | &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Clasificación        
| :- |:- | :-
|[TC018](https://drive.google.com/open?id=14adFusPK1sqlF9hLEqKl86HMZ03XSFzf&disco=AAAADH081mY)| Resolución espacial| ESENCIAL

In [10]:
#####################################################
# TC018_image = datos[Tabla_Resumen[Tabla_Resumen["Z"] == 20].index[0]]
# TC018_image_Res = datos[Tabla_Resumen[Tabla_Resumen["Z"] == -10].index[0]]
#####################################################

In [11]:
TC018_Resolution = TC018_image[0].PixelSpacing[0]
TC018_accums, TC018_cx, TC018_cy, TC018_radii = BusquedaCirculos(
    TC018_image[0].pixel_array, TC018_HoughParameters, TC018_Resolution
)

TC018_cx, TC018_cy

TC018_DistanceMap = np.arange(
    TC018_Limits[0] / TC018_Resolution,
    TC018_Limits[1] / TC018_Resolution,
    TC018_OverResolution,
)

In [12]:
TC018_WaterMask = createCircularMask(
    TC018_image[0].pixel_array.shape[0],
    TC018_image[0].pixel_array.shape[1],
    center=(TC018_cx[0], TC018_cy[0]),
    radius=int(Catphan_ExtRadius * 0.9 / TC018_Resolution)
    * np.logical_not(
        createCircularMask(
            TC018_image[0].pixel_array.shape[0],
            TC018_image[0].pixel_array.shape[1],
            center=(TC018_cx[0], TC018_cy[0]),
            radius=int(Catphan_ExtRadius * 0.8 / TC018_Resolution),
        )
    ),
)
TC018_AirMask = createCircularMask(
    TC018_image[0].pixel_array.shape[0],
    TC018_image[0].pixel_array.shape[1],
    center=(TC018_cx[0], TC018_cy[0]),
    radius=int(Catphan_ExtRadius * 1.3 / TC018_Resolution)
    * np.logical_not(
        createCircularMask(
            TC018_image[0].pixel_array.shape[0],
            TC018_image[0].pixel_array.shape[1],
            center=(TC018_cx[0], TC018_cy[0]),
            radius=int(Catphan_ExtRadius * 1.1 / TC018_Resolution),
        )
    ),
)

HUAgua = TC018_WaterMask * TC018_image[0].pixel_array

HUAire = TC018_AirMask * TC018_image[0].pixel_array


TC018_Angulo = math.sqrt(2 * TC018_Resolution / Catphan_ExtRadius)  # En radianes
TC018_FrecuencyResolution = (
    0.01 / TC018_Resolution
)  # Resolución en frecuencia tomada según IEC62220-1
TC018_FrecuenciaNyquist = 1 / 2.0 / TC018_Resolution
TC018_Oversampling = int(1 / math.tan(TC018_Angulo))  # factor para el "overfitting"
TC018_NumeroDePuntos = int(100 * TC018_Oversampling)

In [13]:
TC018_comp = (
    math.pi
    / 2
    / TC018_FrecuenciaNyquist
    * np.arange(0, TC018_FrecuenciaNyquist, TC018_FrecuencyResolution)
) / np.sin(
    math.pi
    / 2
    / TC018_FrecuenciaNyquist
    * np.arange(0, TC018_FrecuenciaNyquist, TC018_FrecuencyResolution)
)

TC018_image_PA = np.array([X.pixel_array for X in TC018_image])

In [14]:
TC018_Limites = {
    ("Y", "Sup", "Min"): np.max(
        [0, TC018_cy[0] - int(Catphan_ExtRadius / TC018_Resolution) - 50]
    ),
    ("Y", "Inf", "Max"): np.min(
        [
            TC018_image_PA.shape[0] - 1,
            TC018_cy[0] + int(Catphan_ExtRadius / TC018_Resolution) + 50,
        ]
    ),
    ("X", "Izq", "Min"): np.max(
        [0, TC018_cx[0] - int(Catphan_ExtRadius / TC018_Resolution) - 50]
    ),
    ("X", "Dch", "Max"): np.min(
        [
            TC018_image_PA.shape[1] - 1,
            TC018_cx[0] + int(Catphan_ExtRadius / TC018_Resolution) + 50,
        ]
    ),
}

In [15]:
TC018_Data_Y = np.array(
    [
        [
            X[
                TC018_Limites["X", "Izq", "Min"] : TC018_Limites["X", "Izq", "Min"]
                + 100,
                TC018_cy[0] : TC018_cy[0] + TC018_Oversampling,
            ],
            X[
                TC018_Limites["X", "Izq", "Min"] : TC018_Limites["X", "Izq", "Min"]
                + 100,
                TC018_cy[0] - TC018_Oversampling : TC018_cy[0],
            ],
            X[
                TC018_Limites["X", "Dch", "Max"]
                - 100 : TC018_Limites["X", "Dch", "Max"],
                TC018_cy[0] : TC018_cy[0] + TC018_Oversampling,
            ],
            X[
                TC018_Limites["X", "Dch", "Max"]
                - 100 : TC018_Limites["X", "Dch", "Max"],
                TC018_cy[0] - TC018_Oversampling : TC018_cy[0],
            ],
        ]
        for X in TC018_image_PA
    ]
)

In [16]:
TC018_Data_X = np.array(
    [
        [
            X[
                TC018_Limites["X", "Izq", "Min"] : TC018_Limites["X", "Izq", "Min"]
                + 100,
                TC018_cy[0] : TC018_cy[0] + TC018_Oversampling,
            ],
            X[
                TC018_Limites["X", "Izq", "Min"] : TC018_Limites["X", "Izq", "Min"]
                + 100,
                TC018_cy[0] - TC018_Oversampling : TC018_cy[0],
            ],
            X[
                TC018_Limites["X", "Dch", "Max"]
                - 100 : TC018_Limites["X", "Dch", "Max"],
                TC018_cy[0] : TC018_cy[0] + TC018_Oversampling,
            ],
            X[
                TC018_Limites["X", "Dch", "Max"]
                - 100 : TC018_Limites["X", "Dch", "Max"],
                TC018_cy[0] - TC018_Oversampling : TC018_cy[0],
            ],
        ]
        for X in TC018_image_PA
    ]
)

for X in TC018_Data_X:
    X[0] = np.flip(X[0], axis=0)
    X[3] = np.flip(X[3], axis=0)

In [17]:
TC018_Data_Y1 = np.reshape(
    TC018_Data_Y,
    (
        TC018_Data_Y.shape[0] * TC018_Data_Y.shape[1],
        TC018_Data_Y.shape[2] * TC018_Data_Y.shape[3],
    ),
)

In [18]:
TC018_Data_X1 = np.reshape(
    TC018_Data_X,
    (
        TC018_Data_X.shape[0] * TC018_Data_X.shape[1],
        TC018_Data_X.shape[2] * TC018_Data_X.shape[3],
    ),
)

In [19]:
TC018_Data_X1 = np.array(
    [np.reshape(X, (X.shape[0] * X.shape[1])) for Y in TC018_Data_X for X in Y]
)

In [20]:
TC018_Data_Y1 = [
    NormalizeMTF(X, HUAgua[HUAgua > 0.01].mean(), HUAire[HUAire > 0.01].mean())
    for X in TC018_Data_Y1
]

In [21]:
TC018_Data_X1 = [
    NormalizeMTF(X, HUAgua[HUAgua > 0.01].mean(), HUAire[HUAire > 0.01].mean())
    for X in TC018_Data_X1
]

In [22]:
def GetMTF(X):
    return np.absolute(fftpack.fft(np.gradient(X)))

In [23]:
TC018_MTFY = np.array([GetMTF(X) for X in TC018_Data_Y1])
TC018_MTFX = np.array([GetMTF(X) for X in TC018_Data_X1])

In [24]:
def Suavizado(MTF):
    MTF1 = [(MTF[i - 1] + MTF[i] + MTF[i + 1]) / 3 for i in range(1, 49)]
    MTF1.insert(0, 1)
    MTF1.insert(49, (MTF[48] + MTF[49]) / 2)
    MTF1 = np.array(MTF1)
    return MTF1


TC018_MTFY = np.array([Suavizado(X) for X in TC018_MTFY])
TC018_MTFX = np.array([Suavizado(X) for X in TC018_MTFX])

In [25]:
TC018_MTFX_Data = np.around(
    np.vstack([np.arange(0, 10, 0.2), TC018_MTFX.mean(axis=0)[0:50]]).transpose(), 4
)
TC018_MTFY_Data = np.around(
    np.vstack([np.arange(0, 10, 0.2), TC018_MTFY.mean(axis=0)[0:50]]).transpose(), 4
)

In [26]:
def TC018_CalcularMTF(X, data):
    return np.array(
        [
            np.interp(x, np.sort(data.mean(axis=0)[0:50]), np.arange(10, 0, -0.2))
            for x in X
        ]
    )

In [27]:
TC018_SemaforosX = np.around(
    TC018_CalcularMTF(np.array([0.5, 0.1, 0.02]), TC018_MTFX), 2
)
TC018_SemaforosY = np.around(
    TC018_CalcularMTF(np.array([0.5, 0.1, 0.02]), TC018_MTFY), 2
)


def round_up_to_odd(f):
    return (np.ceil(f * 10) // 2 * 2 - 2) / 10

In [28]:
TC018_SemaforosX = round_up_to_odd(TC018_SemaforosX)
TC018_SemaforosY = round_up_to_odd(TC018_SemaforosY)

In [29]:
dataX = [{"x": X[0], "y": X[1], "marker": {"radius": 1}} for X in TC018_MTFX_Data]
dataY = [{"x": X[0], "y": X[1], "marker": {"radius": 1}} for X in TC018_MTFY_Data]
data = [
    {"x": X[0], "y": X[1], "marker": {"radius": 1}}
    for X in (TC018_MTFY_Data + TC018_MTFX_Data) / 2
]

In [30]:
for X in range(len(data)):
    rojo = True
    amarillo = True
    verde = True
    if data[X]["x"] == TC018_SemaforosX[0]:
        data[X]["marker"] = {"symbol": TrafficLightGreen}
    elif data[X]["x"] == TC018_SemaforosX[1]:
        data[X]["marker"] = {
            # "symbol": "url(http://localhost:8888/tree/Resources/Yellow.png)"
            "symbol": TrafficLightYellow
        }
    elif data[X]["x"] == TC018_SemaforosX[2]:
        data[X]["marker"] = {
            # "symbol": "url(http://localhost:8888/tree/Resources/Red.png)"
            "symbol": TrafficLightRed
        }

In [31]:
def TC018_OpcionesGraf():
    options = {
        "chart": {"zoomType": "xy"},
        "title": {"text": "MTF"},
        "xAxis": {
            "labels": {
                "useHTML": True,
                "format": "{value} cm<sup>-1</sup>",
                "enabled": True,
            },
            "title": {
                "text": "Frecuencia espacial",
                "style": {"color": "Highcharts.getOptions().colors[1]"},
            },
        },
        "yAxis": [
            {
                "labels": {
                    "format": "{value}",
                    "style": {
                        "color": "Highcharts.getOptions().colors[1]",
                        "step": 150,
                    },
                },
                "title": {
                    "text": "Función de trasferencia de modulación",
                    "style": {"color": "Highcharts.getOptions().colors[1]"},
                },
            }
        ],
        "tooltip": {"shared": True, "valueDecimals": 1},
    }

    return options


TC018_H = Highchart(width=800, height=600)
TC018_H.set_dict_options(TC018_OpcionesGraf())
TC018_H.add_data_set(
    dataX,
    "spline",
    "MTFX",
    xAxis=0,
    yAxis=0,
    tooltip={
        "headerFormat": "frecuencia: {point.x:.2f} cm <sup>-1</sup><br>",
        "pointFormat": '<span style="font-weight: bold; color: {series.color}">{series.name}</span>: <b>{point.y:.4f}</b><br>',
    },
)
TC018_H.add_data_set(
    dataY,
    "spline",
    "MTFY",
    xAxis=0,
    yAxis=0,
    tooltip={
        "headerFormat": "frecuencia: {point.x:.2f} cm <sup>-1</sup><br>",
        "pointFormat": '<span style="font-weight: bold; color: {series.color}">{series.name}</span>: <b>{point.y:.4f}</b><br>',
    },
)
TC018_H.add_data_set(
    data,
    "spline",
    "MTF",
    xAxis=0,
    yAxis=0,
    tooltip={
        "headerFormat": "frecuencia: {point.x:.2f} cm <sup>-1</sup><br>",
        "pointFormat": '<span style="font-weight: bold; color: {series.color}">{series.name}</span>: <b>{point.y:.4f}</b>',
    },
)

display(TC018_H)

In [32]:
pd.DataFrame(
    [[TC018_SemaforosX[0], TC018_SemaforosX[1], TC018_SemaforosX[2]]],
    columns=["0.5", "0.1", "0.02"],
)

,0.5,0.1,0.02
0,4.0,7.6,9.8
